In [0]:
from silver.Transform_Functions.Trim import func_trim_col
from pyspark.sql import functions as F
from pyspark.sql.functions import trim,col,length

# READING FROM BRONZE TABLE

In [0]:
df = spark.table("workspace.bronze.erp_px_cat_g1v1_raw")

In [0]:
df.display()

# Data Transformation

## Trim whitespaces

In [0]:
df = func_trim_col(df)

## Data Type - Maintenance Flag to Boolean

In [0]:
df = df.withColumn(
    "maintenance",
    F.when(F.upper(col("maintenance")) == "YES", F.lit(True))
     .when(F.upper(col("maintenance")) == "NO", F.lit(False))
     .otherwise(None)
)

## Renaming the columns

In [0]:
erpProdCat_table_header = {
    "ID": "category_id",
    "CAT": "category",
    "SUBCAT": "subcategory",
    "maintenance": "maintenance_flag"
    }


In [0]:
for old_header, new_header in erpProdCat_table_header.items():
    df = df.withColumnRenamed(old_header, new_header)

In [0]:
df.display()

# Write into Silver Schema

In [0]:
(
    df.write.mode("overwrite").format("delta").saveAsTable("silver.erp_product_category")
)


In [0]:
%sql
SELECT * FROM silver.erp_product_category
LIMIT 5;